# Regression - Dummy- GLOB/SEAA/CARB

## Notebook Setup

### Import Libraries

In [ ]:
# Import Standard Libraries
import os
import datetime
import pickle
import pandas as pd
import numpy as np

# Import Visualization Libraries
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import shap

In [ ]:
# Import Modeling Libraries
from sklearn import set_config
from sklearn.inspection import permutation_importance
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, anneal
from hyperopt.early_stop import no_progress_loss
import neptune

In [ ]:
# Pandas Configs
pd.set_option('mode.chained_assignment', None)
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# Ignore Warnings
import warnings
warnings.simplefilter('ignore', category=FutureWarning)
warnings.simplefilter('ignore', category=UserWarning)
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

# MapBox Token for Plotly Maps
px.set_mapbox_access_token(os.environ.get("MAPBOX_TOKEN"))

# Scikit Learn Configs
set_config(transform_output="pandas")

### Import Data

In [ ]:
# GLOB Data
X_train_trans_GLOB = pd.read_parquet("/work/data/Xy_Data/X_train_trans_GLOB.parquet")
X_val_trans_GLOB = pd.read_parquet("/work/data/Xy_Data/X_val_trans_GLOB.parquet")
X_holdout_trans_GLOB = pd.read_parquet("/work/data/Xy_Data/X_holdout_trans_GLOB.parquet")

y_train_GLOB = pd.read_parquet("/work/data/Xy_Data/y_train_GLOB.parquet")
y_val_GLOB = pd.read_parquet("/work/data/Xy_Data/y_val_GLOB.parquet")
y_holdout_GLOB = pd.read_parquet("/work/data/Xy_Data/y_holdout_GLOB.parquet")

# CARB Data
X_train_trans_CARB = pd.read_parquet("/work/data/Xy_Data/X_train_trans_CARB.parquet")
X_val_trans_CARB = pd.read_parquet("/work/data/Xy_Data/X_val_trans_CARB.parquet")
X_holdout_trans_CARB = pd.read_parquet("/work/data/Xy_Data/X_holdout_trans_CARB.parquet")

y_train_CARB = pd.read_parquet("/work/data/Xy_Data/y_train_CARB.parquet")
y_val_CARB = pd.read_parquet("/work/data/Xy_Data/y_val_CARB.parquet")
y_holdout_CARB = pd.read_parquet("/work/data/Xy_Data/y_holdout_CARB.parquet")

# SEAA Data
X_train_trans_SEAA = pd.read_parquet("/work/data/Xy_Data/X_train_trans_SEAA.parquet")
X_val_trans_SEAA = pd.read_parquet("/work/data/Xy_Data/X_val_trans_SEAA.parquet")
X_holdout_trans_SEAA = pd.read_parquet("/work/data/Xy_Data/X_holdout_trans_SEAA.parquet")

y_train_SEAA = pd.read_parquet("/work/data/Xy_Data/y_train_SEAA.parquet")
y_val_SEAA = pd.read_parquet("/work/data/Xy_Data/y_val_SEAA.parquet")
y_holdout_SEAA = pd.read_parquet("/work/data/Xy_Data/y_holdout_SEAA.parquet")

## Dummy Regression Models

In [ ]:
def dummy_model_run(model, X_train_trans, y_train, X_val_trans, y_val):

    model.fit(X_train_trans, y_train) # fit
    predictions = model.predict(X_val_trans) # predict
    mae = mean_absolute_error(y_val, predictions) #score
    deltas = y_val - predictions.reshape(-1, 1)

    return mae, predictions, deltas

In [ ]:
quantile_25_model = DummyRegressor(strategy='quantile', quantile=0.25)
quantile_75_model = DummyRegressor(strategy='quantile', quantile=0.75)
median_model = DummyRegressor(strategy='median')
mean_model = DummyRegressor(strategy='mean')

In [ ]:
# GLOB
mae_quant25_GLOB, predictions_quant25_GLOB, delta_quant25_GLOB = dummy_model_run(quantile_25_model, X_train_trans_GLOB, y_train_GLOB, X_val_trans_GLOB, y_val_GLOB)
mae_quant75_GLOB, predictions_quant75_GLOB, delta_quant75_GLOB = dummy_model_run(quantile_75_model, X_train_trans_GLOB, y_train_GLOB, X_val_trans_GLOB, y_val_GLOB)
mae_median_GLOB, predictions_median_GLOB, delta_median_GLOB = dummy_model_run(median_model, X_train_trans_GLOB, y_train_GLOB, X_val_trans_GLOB, y_val_GLOB)
mae_mean_GLOB, predictions_mean_GLOB, delta_mean_GLOB = dummy_model_run(mean_model, X_train_trans_GLOB, y_train_GLOB, X_val_trans_GLOB, y_val_GLOB)

print(f"Quantile 25 Mean Absolute Error: {mae_quant25_GLOB:.4f}")
print(f"Quantile 75 Mean Absolute Error: {mae_quant75_GLOB:.4f}")
print(f"Median Mean Absolute Error: {mae_median_GLOB:.4f}")
print(f"Mean Mean Absolute Error: {mae_mean_GLOB:.4f}")

In [ ]:
# SEAA
mae_quant25_SEAA, predictions_quant25_SEAA, delta_quant25_SEAA = dummy_model_run(quantile_25_model, X_train_trans_SEAA, y_train_SEAA, X_val_trans_SEAA, y_val_SEAA)
mae_quant75_SEAA, predictions_quant75_SEAA, delta_quant75_SEAA = dummy_model_run(quantile_75_model, X_train_trans_SEAA, y_train_SEAA, X_val_trans_SEAA, y_val_SEAA)
mae_median_SEAA, predictions_median_SEAA, delta_median_SEAA = dummy_model_run(median_model, X_train_trans_SEAA, y_train_SEAA, X_val_trans_SEAA, y_val_SEAA)
mae_mean_SEAA, predictions_mean_SEAA, delta_mean_SEAA = dummy_model_run(mean_model, X_train_trans_SEAA, y_train_SEAA, X_val_trans_SEAA, y_val_SEAA)

print(f"Quantile 25 Mean Absolute Error: {mae_quant25_SEAA:.4f}")
print(f"Quantile 75 Mean Absolute Error: {mae_quant75_SEAA:.4f}")
print(f"Median Mean Absolute Error: {mae_median_SEAA:.4f}")
print(f"Mean Mean Absolute Error: {mae_mean_SEAA:.4f}")

In [ ]:
# CARB
mae_quant25_CARB, predictions_quant25_CARB, delta_quant25_CARB = dummy_model_run(quantile_25_model, X_train_trans_CARB, y_train_CARB, X_val_trans_CARB, y_val_CARB)
mae_quant75_CARB, predictions_quant75_CARB, delta_quant75_CARB = dummy_model_run(quantile_75_model, X_train_trans_CARB, y_train_CARB, X_val_trans_CARB, y_val_CARB)
mae_median_CARB, predictions_median_CARB, delta_median_CARB = dummy_model_run(median_model, X_train_trans_CARB, y_train_CARB, X_val_trans_CARB, y_val_CARB)
mae_mean_CARB, predictions_mean_CARB, delta_mean_CARB = dummy_model_run(mean_model, X_train_trans_CARB, y_train_CARB, X_val_trans_CARB, y_val_CARB)

print(f"Quantile 25 Mean Absolute Error: {mae_quant25_CARB:.4f}")
print(f"Quantile 75 Mean Absolute Error: {mae_quant75_CARB:.4f}")
print(f"Median Mean Absolute Error: {mae_median_CARB:.4f}")
print(f"Mean Mean Absolute Error: {mae_mean_CARB:.4f}")

## Analyzing the Data

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(12, 12))

deltas = {'delta_quant25_GLOB': delta_quant25_GLOB, 'delta_quant75_GLOB': delta_quant75_GLOB, 'delta_median_GLOB': delta_median_GLOB,'delta_mean_GLOB': delta_mean_GLOB,
          'delta_quant25_SEAA': delta_quant25_SEAA, 'delta_quant75_SEAA': delta_quant75_SEAA, 'delta_median_SEAA': delta_median_SEAA,'delta_mean_SEAA': delta_mean_SEAA,
          'delta_quant25_CARB': delta_quant25_CARB, 'delta_quant75_CARB': delta_quant75_CARB, 'delta_median_CARB': delta_median_CARB,'delta_mean_CARB': delta_mean_CARB}

colors = ['blue', 'green', 'red']
alpha = 0.3

counter = 0
for i in range(3):
    for j in range(4):

        df_name = list(deltas.keys())[counter]
        df = deltas[df_name].values.ravel()
        ax = sns.histplot(x=df, ax=axes[i, j], color=colors[i], alpha=alpha, bins=20, kde=True)
        axes[i, j].set_title(df_name)

        counter += 1

plt.tight_layout()
plt.show()